# Sprint13 TensorFlow

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from sklearn.model_selection import train_test_split

# 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

*   重みを初期化する必要があった
*   エポックのループが必要だった
*   バッチ処理
*   バックプロパゲーション
*   クロスエントロピー誤差による損失の計算

# 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

＜回答＞
*   重みを初期化する必要があった
→シェイプを整え、ランダム値で設定
*   エポックのループが必要だった
→for文で記載
*   バッチ処理
→エポックの中のさらなるループで処理
*   バックプロパゲーション
→不明
*   クロスエントロピー誤差による損失の計算
→フォワードプロパゲーションの後で計算

In [142]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
from sklearn.datasets import load_iris

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # 重みとバイアスの宣言
        self.w1 = tf.Variable(tf.random.normal([n_input, n_hidden1]), trainable=True)
        self.w2 = tf.Variable(tf.random.normal([n_hidden1, n_hidden2]), trainable=True)
        self.w3 = tf.Variable(tf.random.normal([n_hidden2, n_classes]), trainable=True)
        self.b1 = tf.Variable(tf.random.normal([n_hidden1]), trainable=True)
        self.b2 = tf.Variable(tf.random.normal([n_hidden2]), trainable=True)
        self.b3 = tf.Variable(tf.random.normal([n_classes]), trainable=True)
    def call(self, x):
        """
        単純な3層ニューラルネットワーク
        """
        layer_1 = tf.add(tf.matmul(x, self.w1), self.b1)
        layer_1 = tf.nn.relu(layer_1)
        layer_2 = tf.add(tf.matmul(layer_1, self.w2), self.b2)
        layer_2 = tf.nn.relu(layer_2)
        layer_output = tf.matmul(layer_2, self.w3) + self.b3  # tf.addと+は等価である
        return layer_output


def train(x, y):
    logits = model(x, training=True)
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(y, logits))#2値分類特有
    return loss

def evaluate(x, y):
    logits = model(x)
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(y, logits)) #2値分類特有
    # 推定結果
    correct_pred = tf.equal(tf.sign(y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
    # 指標値計算
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return loss, accuracy

################
#メイン処理
################

# データセットの読み込み
iris_dataset = load_iris()

temp_X = iris_dataset.data
temp_y = iris_dataset.target.reshape(-1,1)

iris_marged = np.concatenate((temp_X, temp_y), axis=1)
iris_marged = iris_marged[iris_marged[:,4] >= 1]

iris_marged[:,4] = np.where(iris_marged[:,4] == 1, 0, 1)

X = iris_marged[:, 0:4].astype(np.float32)
y = iris_marged[:, 4]
y = y.astype(np.float32).reshape(-1,1)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


#インスタンス化
model = MyModel()


# # 最適化手法
optimizer = tf.keras.optimizers.Adam(learning_rate)


# 計算グラフの実行
for epoch in range(num_epochs):
    # エポックごとにループ
    total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
    total_loss = 0
    total_acc = 0

    for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
        # ミニバッチごとにループ
        with tf.GradientTape() as tape:
            loss = train(mini_batch_x, mini_batch_y)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += loss

    loss = total_loss / n_samples
    val_loss, val_acc = evaluate(X_val, y_val)

    print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, val_acc : {:.3f}".format(epoch, loss, val_loss, val_acc))

_, test_acc = evaluate(X_test, y_test)
print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 14.9768, val_loss : 93.8475, val_acc : 0.375
Epoch 1, loss : 5.7823, val_loss : 15.9875, val_acc : 0.625
Epoch 2, loss : 2.0068, val_loss : 8.2886, val_acc : 0.375
Epoch 3, loss : 0.2590, val_loss : 4.1528, val_acc : 0.375
Epoch 4, loss : 0.0938, val_loss : 0.8891, val_acc : 0.750
Epoch 5, loss : 0.1017, val_loss : 0.0776, val_acc : 0.938
Epoch 6, loss : 0.0653, val_loss : 2.2378, val_acc : 0.750
Epoch 7, loss : 0.1921, val_loss : 4.7458, val_acc : 0.688
Epoch 8, loss : 0.3616, val_loss : 4.4804, val_acc : 0.688
Epoch 9, loss : 0.4438, val_loss : 2.2927, val_acc : 0.812
test_acc : 0.850


# 【問題3】3種類すべての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類すべてを分類できるモデルを作成してください。

In [143]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # 重みとバイアスの宣言
        self.w1 = tf.Variable(tf.random.normal([n_input, n_hidden1]), trainable=True)
        self.w2 = tf.Variable(tf.random.normal([n_hidden1, n_hidden2]), trainable=True)
        self.w3 = tf.Variable(tf.random.normal([n_hidden2, n_classes]), trainable=True)
        self.b1 = tf.Variable(tf.random.normal([n_hidden1]), trainable=True)
        self.b2 = tf.Variable(tf.random.normal([n_hidden2]), trainable=True)
        self.b3 = tf.Variable(tf.random.normal([n_classes]), trainable=True)
    def call(self, x):
        """
        単純な3層ニューラルネットワーク
        """
        layer_1 = tf.add(tf.matmul(x, self.w1), self.b1)
        layer_1 = tf.nn.relu(layer_1)
        layer_2 = tf.add(tf.matmul(layer_1, self.w2), self.b2)
        layer_2 = tf.nn.relu(layer_2)
        layer_output = tf.matmul(layer_2, self.w3) + self.b3  # tf.addと+は等価である
        return layer_output


def train(x, y):
    logits = model(x, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits, axis=1)) #多値分類の場合はsoftmax（確率算出）
   
    return loss

def evaluate(x, y):
    logits = model(x)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits, axis=1))#多値分類の場合はsoftmax（確率算出）
    
    # 推定結果
    correct_pred = tf.equal( tf.argmax(y, axis=1), tf.argmax(logits, axis=1) )#多値分類の場合はこう（accも）
    # 指標値計算
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return loss, accuracy


# データセットの読み込み
iris_dataset = load_iris()

X = iris_dataset.data.astype(np.float32)
y = iris_dataset.target.astype(np.int32)
y = np.identity(3)[y] #onehot


# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 50
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


#インスタンス化
model = MyModel()


# # 最適化手法
optimizer = tf.keras.optimizers.Adam(learning_rate)


# 計算グラフの実行
for epoch in range(num_epochs):
    # エポックごとにループ
    total_batch = np.ceil(X_train.shape[0] / batch_size).astype(np.int)
    total_loss = 0
    total_acc = 0

    for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
        # ミニバッチごとにループ
        with tf.GradientTape() as tape:
            loss = train(mini_batch_x, mini_batch_y)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += np.sum(loss)

    loss = total_loss / n_samples
    val_loss, val_acc = evaluate(X_val, y_val)

    print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, val_acc : {:.3f}".format(epoch, loss, val_loss, val_acc))

_, test_acc = evaluate(X_test, y_test)
print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 6.8708, val_loss : 5.5243, val_acc : 0.708
Epoch 1, loss : 0.7200, val_loss : 11.9320, val_acc : 0.708
Epoch 2, loss : 0.2627, val_loss : 1.3556, val_acc : 0.833
Epoch 3, loss : 0.0326, val_loss : 1.3494, val_acc : 0.833
Epoch 4, loss : 0.0347, val_loss : 1.8996, val_acc : 0.917
Epoch 5, loss : 0.0181, val_loss : 1.2068, val_acc : 0.958
Epoch 6, loss : 0.0193, val_loss : 1.2095, val_acc : 0.958
Epoch 7, loss : 0.0171, val_loss : 1.4245, val_acc : 0.958
Epoch 8, loss : 0.0150, val_loss : 1.7277, val_acc : 0.917
Epoch 9, loss : 0.0130, val_loss : 1.0894, val_acc : 0.958
Epoch 10, loss : 0.0135, val_loss : 1.0986, val_acc : 0.958
Epoch 11, loss : 0.0126, val_loss : 1.3210, val_acc : 0.958
Epoch 12, loss : 0.0129, val_loss : 2.1461, val_acc : 0.917
Epoch 13, loss : 0.0199, val_loss : 1.3692, val_acc : 0.833
Epoch 14, loss : 0.0821, val_loss : 2.1141, val_acc : 0.833
Epoch 15, loss : 0.0644, val_loss : 1.4303, val_acc : 0.958
Epoch 16, loss : 0.0118, val_loss : 4.0251, val_a

# 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

In [139]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # 重みとバイアスの宣言
        self.w1 = tf.Variable(tf.random.normal([n_input, n_hidden1]), trainable=True)
        self.w2 = tf.Variable(tf.random.normal([n_hidden1, n_hidden2]), trainable=True)
        self.w3 = tf.Variable(tf.random.normal([n_hidden2, n_classes]), trainable=True)
        self.b1 = tf.Variable(tf.random.normal([n_hidden1]), trainable=True)
        self.b2 = tf.Variable(tf.random.normal([n_hidden2]), trainable=True)
        self.b3 = tf.Variable(tf.random.normal([n_classes]), trainable=True)
    def call(self, x):
        """
        単純な3層ニューラルネットワーク
        """
        layer_1 = tf.add(tf.matmul(x, self.w1), self.b1)
        layer_1 = tf.nn.relu(layer_1)
        layer_2 = tf.add(tf.matmul(layer_1, self.w2), self.b2)
        layer_2 = tf.nn.relu(layer_2)
        layer_output = tf.matmul(layer_2, self.w3) + self.b3  # tf.addと+は等価である
        return layer_output


def train(x, y):
    logits = model(x, training=True)
    loss = tf.keras.metrics.mean_squared_error(y, logits)
   
    return loss

def evaluate(x, y):
    logits = model(x)
    loss = tf.keras.metrics.mean_squared_error(y, logits)
    
    # 推定結果
    mae = tf.keras.metrics.mean_absolute_error(y, logits)

    return loss, mae


# データセットの読み込み


import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('drive/My Drive/data/house-prices-advanced-regression-techniques_train.csv')
X = df[['GrLivArea', 'YearBuilt']].values
y = df['SalePrice'].values.reshape(-1,1)

scaler = StandardScaler()
X = scaler.fit_transform(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 100
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


#インスタンス化
model = MyModel()


# # 最適化手法
optimizer = tf.keras.optimizers.Adam(learning_rate)


# 計算グラフの実行
for epoch in range(num_epochs):
    # エポックごとにループ
    total_batch = np.ceil(X_train.shape[0] / batch_size).astype(np.int)
    total_loss = 0
    val_mae_total = 0

    for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
        # ミニバッチごとにループ
        with tf.GradientTape() as tape:
            loss = train(mini_batch_x, mini_batch_y)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += np.mean(loss)

    loss = total_loss
    val_loss, val_mae = evaluate(X_val, y_val)
    val_loss = np.mean(val_loss)
    val_mae_total += np.mean(val_mae)

    print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, val_mae : {:.3f}".format(epoch, loss, val_loss, val_mae_total))

_, test_mae = evaluate(X_test, y_test)
test_mae = np.mean(test_mae)
print("test_mae : {:.3f}".format(test_mae))

Epoch 0, loss : 388452843520.0000, val_loss : 37070831616.0000, val_mae : 177701.328
Epoch 1, loss : 387762069504.0000, val_loss : 36991324160.0000, val_mae : 177474.562
Epoch 2, loss : 386795079680.0000, val_loss : 36870176768.0000, val_mae : 177138.531
Epoch 3, loss : 385304393728.0000, val_loss : 36681842688.0000, val_mae : 176624.578
Epoch 4, loss : 383012239360.0000, val_loss : 36396298240.0000, val_mae : 175852.312
Epoch 5, loss : 379580866560.0000, val_loss : 35976261632.0000, val_mae : 174720.484
Epoch 6, loss : 374608404480.0000, val_loss : 35379175424.0000, val_mae : 173110.781
Epoch 7, loss : 367650959360.0000, val_loss : 34558554112.0000, val_mae : 170890.844
Epoch 8, loss : 358235131904.0000, val_loss : 33468020736.0000, val_mae : 167916.172
Epoch 9, loss : 345907888128.0000, val_loss : 32067313664.0000, val_mae : 164044.062
Epoch 10, loss : 330310803456.0000, val_loss : 30327226368.0000, val_mae : 159141.094
Epoch 11, loss : 311211890688.0000, val_loss : 28234629120.0000,

# 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [138]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # 重みとバイアスの宣言
        self.w1 = tf.Variable(tf.random.normal([n_input, n_hidden1]), trainable=True)
        self.w2 = tf.Variable(tf.random.normal([n_hidden1, n_hidden2]), trainable=True)
        self.w3 = tf.Variable(tf.random.normal([n_hidden2, n_classes]), trainable=True)
        self.b1 = tf.Variable(tf.random.normal([n_hidden1]), trainable=True)
        self.b2 = tf.Variable(tf.random.normal([n_hidden2]), trainable=True)
        self.b3 = tf.Variable(tf.random.normal([n_classes]), trainable=True)
    def call(self, x):
        """
        単純な3層ニューラルネットワーク
        """
        layer_1 = tf.add(tf.matmul(x, self.w1), self.b1)
        layer_1 = tf.nn.relu(layer_1)
        layer_2 = tf.add(tf.matmul(layer_1, self.w2), self.b2)
        layer_2 = tf.nn.relu(layer_2)
        layer_output = tf.matmul(layer_2, self.w3) + self.b3  # tf.addと+は等価である
        return layer_output


def train(x, y):
    logits = model(x, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits, axis=1)) #多値分類の場合はsoftmax（確率算出）
   
    return loss

def evaluate(x, y):
    logits = model(x)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits, axis=1))#多値分類の場合はsoftmax（確率算出）
    
    # 推定結果
    correct_pred = tf.equal( tf.argmax(y, axis=1), tf.argmax(logits, axis=1) )#多値分類の場合はこう（accも）
    # 指標値計算
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return loss, accuracy


# データセットの読み込み
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 784).astype(np.float32)
X_test = X_test.reshape(-1, 784).astype(np.float32)

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)



# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 100
num_epochs = 5
n_hidden1 = 400
n_hidden2 = 200
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)


#インスタンス化
model = MyModel()


# # 最適化手法
optimizer = tf.keras.optimizers.Adam(learning_rate)


# 計算グラフの実行
for epoch in range(num_epochs):
    # エポックごとにループ
    total_batch = np.ceil(X_train.shape[0] / batch_size).astype(np.int)
    total_loss = 0
    total_acc = 0

    for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
        # ミニバッチごとにループ
        with tf.GradientTape() as tape:
            loss = train(mini_batch_x, mini_batch_y)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += np.sum(loss)

    loss = total_loss / n_samples
    val_loss, val_acc = evaluate(X_val, y_val)

    print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, val_acc : {:.3f}".format(epoch, loss, val_loss, val_acc))

_, test_acc = evaluate(X_test, y_test_one_hot)
print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 124.3265, val_loss : 3192.2466, val_acc : 0.914
Epoch 1, loss : 16.8057, val_loss : 1806.1128, val_acc : 0.925
Epoch 2, loss : 7.8272, val_loss : 1396.7643, val_acc : 0.929
Epoch 3, loss : 4.9731, val_loss : 1111.8914, val_acc : 0.938
Epoch 4, loss : 3.3195, val_loss : 1040.7947, val_acc : 0.938
test_acc : 0.937


モジュール版

In [5]:
#
#データの準備
#

mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True) #データ読み込み
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test'] #トレインテストスプリット

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples #トレインから10%のバリデーション作成
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

#0〜255を0と1の間に縮める（演算スピードが上がる、計算精度が上がる）
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

buffer_size = 10000 #メモリに気を使い、ちょっとずつ処理を進める
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size) #シャッフル

#トレインとバリデーションの作成
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples) #skip=引数のデータを除く

batch_size = 100

#トレインのみバッチを回す（テストとバリデーションはフォワードのみのため不要）
train_data = train_data.batch(batch_size) 
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [6]:
#モデルの作成

input_size = 784 #28*28
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
])

In [7]:
#最適化アルゴリズムと損失関数の決定
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
#学習
num_epochs = 5
validation_steps = num_validation_samples #バッチサイズ（バリデーションはバッチに分けず一回で実施のため）

model.fit(train_data, epochs=num_epochs, validation_data=(validation_inputs, validation_targets), validation_steps=validation_steps, verbose=2)

Epoch 1/5
540/540 - 13s - loss: 0.4106 - accuracy: 0.8843 - val_loss: 0.2050 - val_accuracy: 0.9413
Epoch 2/5
540/540 - 8s - loss: 0.1897 - accuracy: 0.9456 - val_loss: 0.1656 - val_accuracy: 0.9492
Epoch 3/5
540/540 - 8s - loss: 0.1431 - accuracy: 0.9576 - val_loss: 0.1340 - val_accuracy: 0.9587
Epoch 4/5
540/540 - 9s - loss: 0.1175 - accuracy: 0.9646 - val_loss: 0.1072 - val_accuracy: 0.9678
Epoch 5/5
540/540 - 9s - loss: 0.0994 - accuracy: 0.9705 - val_loss: 0.0969 - val_accuracy: 0.9710


In [9]:
#推定
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.1111 - accuracy: 0.9679
